In [6]:
import os
import cv2
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from pathlib import Path


In [7]:

# Feature extraction functions (unchanged)
def calculate_optical_flow(video_path):
    cap = cv2.VideoCapture(video_path)
    flow_values = []

    ret, prev_frame = cap.read()
    if not ret:
        cap.release()
        return 0, 0

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None,
                                            0.5, 3, 15, 3, 5, 1.2, 0)
        flow_values.append(flow)
        prev_gray = gray
    
    cap.release()
    flow_values = np.array(flow_values)
    return np.mean(flow_values), np.std(flow_values)



In [8]:

def calculate_edge_ratio(video_path):
    cap = cv2.VideoCapture(video_path)
    edge_pixels = 0
    total_pixels = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        edges = cv2.Canny(frame, 100, 200)
        edge_pixels += np.count_nonzero(edges)
        total_pixels += frame.shape[0] * frame.shape[1]
    
    cap.release()
    return edge_pixels / total_pixels if total_pixels != 0 else 0


In [9]:


def count_keypoints(video_path):
    cap = cv2.VideoCapture(video_path)
    sift = cv2.SIFT_create()
    keypoint_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        kp, _ = sift.detectAndCompute(gray, None)
        keypoint_count += len(kp)
    
    cap.release()
    return keypoint_count


In [10]:


def extract_fft_peaks(video_path):
    cap = cv2.VideoCapture(video_path)
    peak_values = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        fft = np.fft.fft2(gray)
        fft_shift = np.fft.fftshift(fft)
        fft_magnitude = np.abs(fft_shift)
        peak_values.append(np.max(fft_magnitude))
    
    cap.release()
    top_peaks = sorted(peak_values, reverse=True)[:3]
    while len(top_peaks) < 3:
        top_peaks.append(0)
    return top_peaks


In [11]:


def extract_features_for_videos(video_paths, state, segment):
    data = []
    for video_path in tqdm(video_paths, desc=f"Processing {state} - {segment}"):
        video_name = os.path.basename(video_path)
        
        # Determine view from filename
        view = 'angle' if 'angle' in video_name.lower() else 'front'

        # Feature extraction
        mean_flow, std_flow = calculate_optical_flow(video_path)
        edge_ratio_value = calculate_edge_ratio(video_path)
        keypoint_count = count_keypoints(video_path)
        fft_peaks = extract_fft_peaks(video_path)

        # Add data for each frame in the segment
        frame_index = 0  # Start at frame 0
        cap = cv2.VideoCapture(video_path)
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            # Extract features per frame
            data.append([video_name, frame_index, mean_flow, std_flow, edge_ratio_value, keypoint_count,
                         *fft_peaks, view, state])
            frame_index += 1
        cap.release()
    
    return data



In [12]:

# Directory setup
base_dir = Path('evm_segmented_videos')
output_dir = Path('Datasets')
states = ['Bearing_fault', 'Normal_state', 'Unbalance_weight']
segments = ['segmented_5', 'segmented_5_overlap', 'segmented_10',
            'segmented_10_overlap', 'segmented_15', 'segmented_15_overlap']

for state in states:
    (output_dir / state).mkdir(parents=True, exist_ok=True)

# Process and generate CSVs
for state in states:
    for segment in segments:
        segment_path = base_dir / state / segment
        video_paths = glob(str(segment_path / "*.avi"))
        if not video_paths:
            continue
        
        # Extract features for all frames in this segment
        segment_data = extract_features_for_videos(video_paths, state, segment)
        
        # Ensure the directory exists before saving
        segment_output_dir = output_dir / state / segment
        segment_output_dir.mkdir(parents=True, exist_ok=True)

        # Save the segment's data to CSV
        if segment_data:
            columns = ['video_name', 'frame_index', 'mean_flow', 'std_flow', 'edge_ratio', 'keypoint_count',
                       'fft_peak1', 'fft_peak2', 'fft_peak3', 'view', 'state']
            df = pd.DataFrame(segment_data, columns=columns)
            output_csv = segment_output_dir / f"{segment}_features.csv"
            df.to_csv(output_csv, index=False)

print("✅ Feature extraction and CSV generation for each time slice completed!")


Processing Unbalance_weight - segmented_5: 100%|██████████| 36/36 [02:01<00:00,  3.37s/it]
Processing Unbalance_weight - segmented_5_overlap: 100%|██████████| 72/72 [03:42<00:00,  3.09s/it]
Processing Unbalance_weight - segmented_10: 100%|██████████| 18/18 [01:42<00:00,  5.67s/it]
Processing Unbalance_weight - segmented_10_overlap: 100%|██████████| 34/34 [03:07<00:00,  5.52s/it]
Processing Unbalance_weight - segmented_15: 100%|██████████| 12/12 [01:46<00:00,  8.89s/it]
Processing Unbalance_weight - segmented_15_overlap: 100%|██████████| 22/22 [02:59<00:00,  8.16s/it]

✅ Feature extraction and CSV generation for each time slice completed!
